In [11]:
import os
from os import listdir

In [8]:
####### STEP 1. set up parameters for mapping #######
# Create a project folder and under /mnt/BioAdHoc/Groups/
# With in the project older, generate a folder called Fastq_Paired, and move all paired-end fastq files here: Note - do not put fastq files in sub folders

thread = 4
# name of samples: unique sample names without appendix "_RX_001.fastq.gz"
samples = list(set(x.split('.')[0][:-7] for x in listdir('./Fastq_Paired/')))
# wehre to export the bash files for submission - relative directory from this notebook
# !! Make sure this dir is located on the server
os.system('mkdir -p Submissions')

## Directory defined below are absolute directory on the server
# directory where the project root is located
dirin = '/mnt/BioAdHoc/Groups/vd-vijay/ndu/RSS_113_AsthmaAirways'
# directory where temperatory files are located
temp_dir = '/mnt/BioScratch/ndu/asthma_airway'
# directory where reference files are located
ref_dir = '/home/ndu/bioinfo/Reference/STAR-GRCh37_gencode_v19'
gtf_dir = '/home/ndu/bioinfo/Reference/GRCh37.p13.gencode.r19/gencode.v19.annotation.gtf'


In [9]:
####### STEP 2. generate bash scripts for submission #######

for sample in samples:
    fastq_f = sample+'_R1_001.fastq.gz'
    fastq_r = sample+'_R2_001.fastq.gz'
    with open(f'Submissions/{sample}.sh','w') as f:
        f.write(f'#!/bin/bash\n#PBS -N {sample}\n#PBS -o {temp_dir}/out_{sample}\n#PBS -e {temp_dir}/err_{sample}\n#PBS -q default\n#PBS -l nodes=1:ppn={thread}\n#PBS -l mem=40gb\n#PBS -l walltime=10:00:00\ncd {dirin}/\nmkdir -p {temp_dir}\nmkdir -p fastp_output\nmkdir -p Fastq_filtered\n\n')
        # fastp
        f.write(f'/mnt/BioHome/ndu/anaconda3/bin/fastp -w {thread} -i {dirin}/Fastq_Paired/{fastq_f} -I {dirin}/Fastq_Paired/{fastq_r} -o {dirin}/Fastq_filtered/{fastq_f} -O {dirin}/Fastq_filtered/{fastq_r} -j {dirin}/fastp_output/{sample}_fastp.json\n')
        # STAR_MAPPING
        f.write(f'mkdir -p {dirin}/bam_aligned/{sample}\n')
        f.write(f'/mnt/BioHome/ndu/anaconda3/bin/STAR --runThreadN {thread} --genomeDir {ref_dir} --sjdbGTFfile {gtf_dir} --readFilesIn {dirin}/Fastq_filtered/{fastq_f} {dirin}/Fastq_filtered/{fastq_r} --readFilesCommand zcat --outFileNamePrefix {dirin}/bam_aligned/{sample}/{sample}_\n')
        

with open('Submissions.sh','w') as f:    
    for submission in listdir('Submissions/'):        
        f.write(f'qsub ./Submissions/{submission}\n')

# Login herman server, go to project folder, and bash Submission.sh      

In [5]:
####### STEP 3. featureCount - wait unit all submitted tasks have been completed #######

thread = 4
with open(f'featurecount.sh','w') as f:
    f.write(f'#!/bin/bash\n#PBS -N count_STAR\n#PBS -o {temp_dir}/out_count_STAR\n#PBS -e {temp_dir}/err_count_STAR\n#PBS -q default\n#PBS -l nodes=1:ppn={thread}\n#PBS -l mem=40gb\n#PBS -l walltime=10:00:00\ncd {dirin}\n\n')
    samples = [f'{dirin}/bam_aligned/{sample}/{sample}_Aligned.out.sam' for sample in os.listdir('./bam_aligned/')]
    samfiles = ' '.join(samples)
    f.write(f'/mnt/BioHome/ndu/anaconda3/bin/featureCounts -T {thread} -s 0 -M -p -B -C -O -t gene -g gene_id -a {gtf_dir} -o count_table.csv {samfiles}')

# -M allow multiple mapping
# -p paired end reads
# -C countChimericFragments If specified, the chimeric fragments (those fragments that have their two ends aligned to different chromosomes) will NOT be counted
# -B requireBothEndsMapped
# -O allowMultiOverlap
# −−donotsort 1/autosort
# -s 0 unstranded

# Login herman server, go to project folder, and qsub featurecount.sh      

In [10]:
####### STEP 4. Multiqc - wait until featureCount has bben completed;  generate table and figures for QC #######
with open(f'multiqc.sh','w') as f:
    f.write(f'#!/bin/bash\n#PBS -N multiqc\n#PBS -o {temp_dir}/out_multiqc\n#PBS -e {temp_dir}/err_multiqc\n#PBS -q default\n#PBS -l nodes=1:ppn={thread}\n#PBS -l mem=40gb\n#PBS -l walltime=10:00:00\ncd {dirin}\n\n')
    f.write('/mnt/BioHome/ndu/anaconda3/bin/multiqc . --cl_config max_table_rows:1000')

# Login herman server, go to project folder, and qsub multiqc.sh 